In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import os
import datetime
import glob


In [2]:
def read_file(file):
    """
    read file line by line
    """
    lines = [line.rstrip() for line in open(file, 'r', encoding='ISO-8859-15')]
    return lines

def find_idx(lines, substring):
    """
    find the line indexes containing a substring
    """
    idx = []
    count = 0
    for line in lines:
        if substring in line:
            idx.append(count)
        count += 1
    return idx

def keep_cols_by_name(df, substring):
    df = df.filter(regex=substring)
    return df


def unstack_double_header(df, col_index):
    """
    transform df with double headers into single header
    """

    # Unstack two headers
    cols = df.iloc[col_index][1::].unique()
    cols = (list(set([col.strip() for col in cols])))

    result = pd.DataFrame()
    for col in cols:

        # get the indexes of the desired columns
        idx = np.where(df.iloc[0] == col)[0]
        # insert first column (key)
        idx = np.insert(idx, 0, 0, axis=0)

        # filter just the desired columns and removes lines from headers
        df_aux = df.loc[2::,idx]
        df_aux.columns = df.loc[1, idx]

        df_aux["new_col"] = col
        result = pd.concat([result, df_aux])

    return result

def stack_same_tables_in_same_file(idxs, lines):
    result = pd.DataFrame()
    for i in range(1, len(idxs)):
        fst_idx=idxs[i-1]+2
        last_idx=idxs[i]-1

        df = pd.DataFrame([line.split(";") for line in lines[fst_idx:last_idx]])
        aux = unstack_double_header(df, 0)
        aux["table"] = lines[fst_idx-2]
        result = pd.concat([result, aux])

        print(f"\t Time: {datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')} - Iteraction: {i} - First idx = {fst_idx} - Last idx = {last_idx} - Shape before unstack: {df.shape} - Shape after unstack: {aux.shape} - Shape after append: {result.shape} ")
    return result

def df_janitor(df):

    # Correct new columns added names
    df = df.rename({"new_col":"Faixa Etaria", "table":"Sexo"}, axis='columns')

    # adjust content of the lines of the column Faixa Etaria
    df["Faixa Etaria"] = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', s)) for s in df["Faixa Etaria"]]

    # adjust content of the lines of the column Sexo
    df["Sexo"] = [re.sub("Sexo Trabalhador = ", '', s) for s in df["Sexo"]]
    df["Sexo"] = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', s)) for s in df["Sexo"]]

    # remove unwanted lines
    to_remove = ['{ñ class}', 'Total', '', 'Seleções vigentes', 'Variável', 'Ano', 'Vínculo Ativo 31/12']
    df = df[~df.iloc[:,0].isin(to_remove)]
    df = df[~df["Faixa Etaria"].isin(to_remove)]
    df = df[~df["Sexo"].isin(to_remove)]

    # transform string nan to effective nan
    df = df.replace("NaN", np.nan)

    # adjust columns names
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]

    # adjust line names
    df.iloc[:,0] = df.iloc[:,0].str.split(':').str[1]

    # remove special chars
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]
    df.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df.columns]

    return df

def load_idadeXsexoXsetor(file):
    lines = read_file(file)
    substring = "Sexo Trabalhador = "
    idxs = find_idx(lines, substring)
    # adding the last line of the file (+1 to be ok with the logic below)
    idxs.append(len(lines)+1)

    # read the file and stack the tables
    result = stack_same_tables_in_same_file(idxs, lines)

    # clean the output
    df = df_janitor(result)

    df["arquivo"] = os.path.split(file)[1]

    return df

In [3]:
path = os.getcwd()
extension = 'csv'
os.chdir(path)
files_csv = glob.glob('*.{}'.format(extension))

df_final = pd.DataFrame()
for i in files_csv:
    aux = load_idadeXsexoXsetor(i)
    df_final = pd.concat([df_final, aux])
    print(f"Time: {datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')} - File {i}")

df_final.to_csv("load_idadeXsexoXsetor.csv")

	 Time: 20240128 17:47:49 - Iteraction: 1 - First idx = 2 - Last idx = 32 - Shape before unstack: (30, 261) - Shape after unstack: (280, 30) - Shape after append: (280, 30) 
	 Time: 20240128 17:47:49 - Iteraction: 2 - First idx = 35 - Last idx = 65 - Shape before unstack: (30, 261) - Shape after unstack: (280, 30) - Shape after append: (560, 30) 
	 Time: 20240128 17:47:49 - Iteraction: 3 - First idx = 68 - Last idx = 104 - Shape before unstack: (36, 261) - Shape after unstack: (340, 30) - Shape after append: (900, 30) 
Time: 20240128 17:47:49 - File idadeXsexoXsetorX2008.csv
	 Time: 20240128 17:47:49 - Iteraction: 1 - First idx = 2 - Last idx = 32 - Shape before unstack: (30, 248) - Shape after unstack: (280, 29) - Shape after append: (280, 29) 
	 Time: 20240128 17:47:49 - Iteraction: 2 - First idx = 35 - Last idx = 65 - Shape before unstack: (30, 248) - Shape after unstack: (280, 29) - Shape after append: (560, 29) 
	 Time: 20240128 17:47:49 - Iteraction: 3 - First idx = 68 - Last idx

In [5]:
df_final


,,extrativa_mineral,prod_mineral_nao_metalico,industria_metalurgica,industria_mecanica,eletrico_e_comunic,material_de_transporte,madeira_e_mobiliario,papel_e_graf,borracha_fumo_couros,...,aloj_comunic,medicos_odontologicos_vet,ensino,administracao_publica,agricultura,_n_class_,total,faixa_etaria,sexo,arquivo
2,Porto Velho - RO,9,136,94,16,0,14,125,45,21,...,669,128,204,2.329,140,0,11.024,18 A 24,Masculino,idadeXsexoXsetorX2008.csv
3,Rio Branco - AC,25,109,39,6,16,9,228,26,31,...,508,38,104,1.056,229,0,7.019,18 A 24,Masculino,idadeXsexoXsetorX2008.csv
4,Manaus - AM,9,208,1.483,1.569,3.629,4.419,184,655,869,...,3.071,272,552,2.769,294,0,45.142,18 A 24,Masculino,idadeXsexoXsetorX2008.csv
5,Boa Vista - RR,5,58,5,5,1,0,22,26,6,...,380,36,126,707,89,0,4.327,18 A 24,Masculino,idadeXsexoXsetorX2008.csv
6,Belém - PA,2,61,126,22,19,90,326,226,79,...,3.125,450,484,4.637,242,0,25.679,18 A 24,Masculino,idadeXsexoXsetorX2008.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,Porto Alegre - RS,0,0,3,2,3,2,2,11,3,...,259,168,835,342,4,NaN,2.081,65 OU MAIS,Feminino,idadeXsexoXsetorX2006.csv
25,Campo Grande - MS,0,0,0,0,0,0,0,0,1,...,33,20,12,172,3,NaN,269,65 OU MAIS,Feminino,idadeXsexoXsetorX2006.csv
26,Cuiabá - MT,0,0,0,0,0,1,0,0,0,...,14,4,15,237,1,NaN,299,65 OU MAIS,Feminino,idadeXsexoXsetorX2006.csv
27,Goiânia - GO,0,0,3,0,0,0,1,0,0,...,64,98,21,486,1,NaN,738,65 OU MAIS,Feminino,idadeXsexoXsetorX2006.csv
